In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset = pd.read_csv("Full-Dataset/processed/url_encoded.csv", index_col=0)
merged_rnn = pd.read_csv("Full-Dataset/processed/merged_RNN.csv", index_col=0)

In [6]:
dataset.columns

Index(['frame.time', 'http.request.full_uri', 'tcp.checksum', 'tcp.options',
       'tcp.payload', 'Attack_type', 'GET', 'POST', 'TRACE', 'HTTP/1.0',
       'HTTP/1.1', 'path', 'query'],
      dtype='object')

In [7]:
dataset.groupby(["Attack_type"]).count()

,frame.time,http.request.full_uri,tcp.checksum,tcp.options,tcp.payload,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,path,query
Attack_type,,,,,,,,,,,,
Password,71035,71035,71035,71035,71035,71035,71035,71035,71035,71035,71035,71035
SQL_injection,4379,4379,4379,4379,4379,4379,4379,4379,4379,4379,4379,4379
Uploading,1907,1907,1907,1907,1907,1907,1907,1907,1907,1907,1907,1907
XSS,1376,1376,1376,1376,1376,1376,1376,1376,1376,1376,1376,1376


In [10]:
all_data = pd.read_csv("Full-Dataset/processed/merged_data.csv")

C:\Users\ParkJaehyung\AppData\Local\Temp\ipykernel_11744\1170755503.py:1: DtypeWarning: Columns (5,8,15,38) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv("Full-Dataset/processed/merged_data.csv")


In [11]:
all_data.columns

Index(['Unnamed: 0', 'ip.src_host', 'frame.time', 'tcp.checksum',
       'http.response', 'arp.src.proto_ipv4', 'http.content_length',
       'Attack_type', 'arp.dst.proto_ipv4', 'tcp.connection.rst',
       'arp.hw.size', 'tcp.len', 'tcp.seq', 'tcp.options',
       'tcp.connection.syn', 'http.referer', 'ip.dst_host', 'tcp.flags.ack',
       'tcp.ack_raw', 'tcp.connection.fin', 'tcp.dstport', 'tcp.flags',
       'tcp.ack', 'http.file_data', 'tcp.connection.synack', 'tcp.payload',
       'tcp.srcport', 'arp.opcode', '0', 'GET', 'POST', 'TRACE', 'HTTP/1.0',
       'HTTP/1.1', 'http.request.uri.scheme', 'http.request.uri.netloc',
       'http.request.uri.path', 'index', 'http.data.key', 'http.data.value'],
      dtype='object')

In [12]:
all_data["Attack_type"].unique()

array(['Password', 'SQL_injection', 'Uploading', 'XSS'], dtype=object)

In [8]:
merged_rnn.columns

Index(['http.data.value', 'http.data.key', 'Attack_type'], dtype='object')

In [5]:
merged_rnn.describe()

,http.data.value,http.data.key,Attack_type
count,79789,79789,79789
unique,3846,5,2
top,admin,username,Password
freq,23677,23677,71031


In [54]:
class DataSet:
    labels: pd.Series
    features: pd.DataFrame | pd.Series
    data_train: pd.DataFrame | pd.Series
    target_train: pd.DataFrame | pd.Series
    data_test: pd.DataFrame | pd.Series
    target_test: pd.DataFrame | pd.Series

    def __init__(self, labels ,features, test_size=.2):
        self.labels = labels
        self.features = features
        self.data_train, self.data_test, self.target_train, self.target_test = train_test_split(self.features, self.labels, test_size=test_size)
    def split_data(self, test_size=.2):
        self.data_train, self.data_test, self.target_train, self.target_test = train_test_split(self.features, self.labels, test_size=test_size)
        return self.data_train, self.data_test, self.target_train, self.target_test

            
merged_RNN = DataSet(merged_rnn["Attack_type"], merged_rnn.copy(deep=True).drop(["Attack_type"], axis="columns"))

In [57]:
merged_RNN.data_train.head(5)

,http.data.value,http.data.key
109609,00,password
113263,Login,Login
59549,00,password
123672,"2) AND EXTRACTVALUE(8862,CONCAT(0x5c,0x7171707...",id
100748,Login,Login


In [58]:
# model for classifying SQL / plain statement / XSS payload
# reference 01: https://wikidocs.net/45101
# reference 02: https://itstory1592.tistory.com/39
# reference 03: https://www.youtube.com/watch?v=IrPhMM_RUmg
# reference 04: https://wikidocs.net/106473
model = keras.Sequential()
model.add(keras.Input(2))
model.add(layers.SimpleRNN(2, activation="relu"))
model.add(layers.Dense(3))
model.summary()

ValueError: Input 0 of layer "simple_rnn" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 2)

In [43]:
model.fit(merged_RNN.data_train, merged_RNN.target_train)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [5]:
merged_rnn.help

'help'

In [24]:
labels, features = dataset["Attack_type"], dataset.copy(deep=True).drop(["Attack_type", "frame.time"], axis="columns")

In [25]:
data_train, data_test, target_train, target_test = train_test_split(features, labels,test_size=0.20)

In [21]:
def getRNN_dataset(df):
    columns = ["http.data.key", "http.data.value", "Attack_type"]
    columns = [_ for _ in columns if _ in df.columns]
    return df[list(columns)]
def get_logistic_dataset(df):
    columns = ["http.data.key", "http.data.value"]
    columns = [_ for _ in columns if _ in df.columns]
    return df.drop(columns, axis=1)

In [31]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# 데이터를 로드합니다.
df = all_data

# LabelEncoder 클래스를 생성합니다.
encoder = LabelEncoder()
df["Attack_type"] = encoder.fit_transform(df["Attack_type"])

# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
splitted_data = train_test_split(df, df["Attack_type"], test_size=0.2)
X_train, X_test, y_train, y_test = splitted_data
RNN_X_train, RNN_X_test = [getRNN_dataset(_) for _ in [X_train, X_test]]
logistic_X_train, logistic_X_test = [get_logistic_dataset(_) for _ in [X_train, X_test]]

In [33]:
encoder.classes_

array(['Password', 'SQL_injection', 'Uploading', 'XSS'], dtype=object)

In [34]:
RNN_X_train.head(10)

,http.data.key,http.data.value,Attack_type
20655,NaN,0,0
57710,Login,Login,0
83902,NaN,0,0
133686,NaN,0,3
61834,NaN,0,0
144273,NaN,"<!DOCTYPE HTML PUBLIC ""-//IETF//DTD HTML 2.0//...",3
36703,NaN,0,0
135099,NaN,0,3
132166,NaN,0.0,2
62791,NaN,0,0


In [36]:
# RNN을 사용하여 요청의 URI 키와 값을 분류하는 모델을 학습합니다.
rnn_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(5000, 128),
  tf.keras.layers.Conv1D(64, 3, padding='same'),
  tf.keras.layers.LSTM(128),
  tf.keras.layers.Dense(4, activation="softmax")
])

rnn_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
rnn_model.fit(RNN_X_train, y_train, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [26]:
# 로지스틱 회귀를 사용하여 나머지 데이터를 분류하는 모델을 학습합니다.
logistic_regression_model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(4, activation="softmax")
])

logistic_regression_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
logistic_regression_model.fit(X_train, y_train, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# 두 모델의 예측을 결합하여 최종 예측을 만듭니다.
predictions = rnn_model.predict(RNN_X_test) * 0.5 + logistic_regression_model.predict(logistic_X_test) * 0.5

# 모델의 성능을 평가합니다.
accuracy = tf.metrics.accuracy(labels=y_test, predictions=predictions)
print("Accuracy:", accuracy)